Pruebe 5 modelos distintos, variando los siguientes hiperparámetros:
- Cantidad de capas Conv
- Cantidad de feature maps (filtros convolucionares)
- Stride, padding y kernel_size de las convoluciones
- Funciones de activación
- Uso de capas Max Pooling.

In [0]:
!pip install keras==2.1.3 tensorflow==1.14.0

     |████████████████████████████████| 327kB 2.9MB/s 
     |████████████████████████████████| 109.2MB 1.2MB/s 


In [12]:
import numpy as np
import tensorflow as tf
import keras
import numpy as np
import pandas as pd
import os
import seaborn as sns
%load_ext autoreload
%autoreload 2
%matplotlib inline

import matplotlib.pyplot as plt
plt.rcParams['image.cmap'] = 'gray'
from sklearn import metrics, preprocessing

import AAPutils
import utils

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [0]:
parameters = pd.read_csv('parameters.csv')
# parameters.pooling_stride = parameters.pooling_stride.astype('int')

In [0]:
from keras.datasets import mnist
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [0]:
scaler = utils.Rescaler()
scaler.fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
X_train = X_train.reshape(-1, 28, 28, 1)
X_test = X_test.reshape(-1, 28, 28, 1)

In [0]:
classes = np.unique(y_train).size
input_shape = X_train[0].shape

Defining model

In [0]:
def create_model(
    feature_maps=10, 
    kernel_size=3, 
    conv_stride=1, 
    max_pooling=True, 
    pooling_stride=None,  
    activation_max_pooling='relu',
    **kwargs):
    print(conv_stride)
    layers = []
    if (feature_maps != None):
        conv2d_layer = keras.layers.Conv2D(
            feature_maps,
            kernel_size=(kernel_size, kernel_size), 
            strides=(conv_stride, conv_stride),
            padding='valid',
            input_shape= input_shape,
            activation=activation_max_pooling,
        )
        
        layers.append(conv2d_layer)
    else:
        raise Exception('El parametro feature maps es obligatorio')
    
    if (max_pooling):
        pooling_stride = None if pooling_stride==0 else pooling_stride
        max_pooling_layer = keras.layers.MaxPool2D(strides=pooling_stride)
        layers.append(max_pooling_layer)
    
    layers.append(keras.layers.Dropout(0.8))
    layers.append(keras.layers.Flatten())
    
    #layer1 = keras.layers.Dense(10, activation='relu')
    layers.append(keras.layers.Dense(classes, activation='softmax'))

    model = keras.models.Sequential(layers)

    model.compile(
      optimizer= 'adam',
      loss='sparse_categorical_crossentropy', 
      metrics=['accuracy'], 
    )
    
    model.summary()
    
    return model

In [19]:
models = [
    create_model(**r) for i,r in parameters.iterrows()
]
    

1





Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 4)         40        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 25, 25, 4)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 25, 25, 4)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2500)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 10)                25010     
Total params: 25,050
Trainable params: 25,050
Non-trainable params: 0
__________________________________________________________

In [0]:
def fit_all(model):
    epochs=1000
    history = model.fit(
        X_train, 
        y_train, 
        epochs=epochs, 
        batch_size=64, 
        verbose=2, 
        callbacks = [
           keras.callbacks.EarlyStopping(
               monitor='val_loss', 
               patience=5,
               verbose=2, 
               mode='auto', 
               restore_best_weights=True
           )
        ],
        validation_data = (X_test, y_test),
        #class_weight = weight # acá va esto? o el de train?
    )
    return history

In [23]:
histories = [
  fit_all(m)  for m in models
]

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



Train on 60000 samples, validate on 10000 samples
Epoch 1/1000





 - 12s - loss: 0.6361 - acc: 0.8016 - val_loss: 0.2250 - val_acc: 0.9393
Epoch 2/1000
 - 5s - loss: 0.3426 - acc: 0.8952 - val_loss: 0.1650 - val_acc: 0.9542
Epoch 3/1000
 - 5s - loss: 0.2987 - acc: 0.9083 - val_loss: 0.1438 - val_acc: 0.9619
Epoch 4/1000
 - 5s - loss: 0.2645 - acc: 0.9182 - val_loss: 0.1269 - val_acc: 0.9654
Epoch 5/1000
 - 5s - loss: 0.2462 - acc: 0.9246 - val_loss: 0.1168 - val_acc: 0.9674
Epoch 6/1000
 - 5s - loss: 0.2304 - acc: 0.9287 - val_loss: 0.1130 - val_acc: 0.9672
Epoch 7/1000
 - 5s - loss: 0.2237 - acc: 0.9304 - val_loss: 0.1051 - val_acc: 0.9695
Epoch 8/1000
 - 5s - loss: 0.2169 - acc: 0.9329 - val_loss: 0.1003 - val_acc: 0.9705
Epoch 9/1000
 - 5s - loss: 0.2100 - acc: 0.9353 - val_loss: 0.0979 - val_acc: 0.9716
Epoch 10/1000
 - 5s - loss: 0.2056 - acc: 0.9361 - val_loss: 0.0966 - val_acc: 0.9

In [25]:
[m.evaluate(X_test, y_test) for m in models]

10000/10000 [==============================] - 1s 82us/step


[[0.07267276928909124, 0.9774],
 [0.04259509892365895, 0.9863],
 [0.04433515391489491, 0.9871],
 [0.04564127496061847, 0.9857],
 [0.05638595878952183, 0.9822],
 [0.03306266460164916, 0.9888],
 [0.035370001608174065, 0.9879],
 [0.039613921019179904, 0.9889],
 [0.054476723199384285, 0.9827],
 [0.08612295546717942, 0.9735],
 [0.07433237283863127, 0.9775],
 [0.048479090684070254, 0.9842]]

In [32]:
output = {'epochs':[], 'acc':[], 'val_acc':[]}
for i in range(len(histories)):
  output['epochs'].append(len(histories[i].history['loss']))
  output['acc'].append(models[i].evaluate(X_train, y_train)[1])
  output['val_acc'].append(models[i].evaluate(X_test, y_test)[1])
pd.DataFrame(output)

10000/10000 [==============================] - 1s 79us/step


,epochs,acc,val_acc
0,53,0.981300,0.9774
1,25,0.992833,0.9863
2,16,0.996150,0.9871
3,12,0.995217,0.9857
4,17,0.983100,0.9822
5,20,0.992767,0.9888
6,10,0.992083,0.9879
7,13,0.994867,0.9889
8,31,0.990850,0.9827
9,38,0.979500,0.9735


In [0]:
pd.DataFrame(output).to_csv('output.csv', header=True)